In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [71]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

with open('../data/raw/spotify_token.json') as f:
    spotify_token = json.load(f)

In [72]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
scope = spotify_creds['saved_library_scope']
redirect_uri = spotify_creds['saved_library_redirect_url']
token = spotify_token['all_access_token']


def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    #token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [52]:
def get_top_artists():
    df_top_artists = pd.DataFrame()
    artists_list = []
    genres_list = []
    artists_pic_list = []
    popularity_list = []
    artist_url_list = []
    followers_list = []
    artists = sp.current_user_top_artists()
    
    for artist in artists['items']:
        artists_pic_list.append(artist['images'][0]['url'])
        artists_list.append(artist['name'])
        genres_list.append(artist['genres'][0])
        popularity_list.append(artist['popularity'])
        followers_list.append(artist['followers']['total'])
        artist_url = artist['external_urls']['spotify']
        artist_url = artist_url.split('/')[-1]
        artist_url_list.append(artist_url)

        
    df_top_artists['artist'] = artists_pic_list 
    df_top_artists['name'] = artists_list
    df_top_artists['genere'] = genres_list
    df_top_artists['popularity'] = popularity_list
    df_top_artists['followers'] = followers_list
    df_top_artists['url'] = artist_url_list

    return df_top_artists

In [53]:
df=get_top_artists()

In [136]:
def get_related_artists(artist_name):
    
    df_related_artists = pd.DataFrame()
    artists_list = []
    genres_list = []
    artists_pic_list = []
    popularity_list = []
    artist_url_list = []
    followers_list = []
    
    artist_id = sp.search(artist_name)
    artist_url = artist_id['tracks']['items'][0]['artists'][0]['external_urls']['spotify']
    artist_url = artist_url.split('/')[-1]
    
    artists = sp.artist_related_artists(artist_url)
    
    for artist in artists['artists']:
        artists_list.append(artist['name'])
        #genres_list.append(artist['genres'][0])
        popularity_list.append(artist['popularity'])
        followers_list.append(artist['followers']['total'])
        artist_url = artist['external_urls']['spotify']
        artist_url = artist_url.split('/')[-1]
        artist_url_list.append(artist_url)

        
    
    df_related_artists['name'] = artists_list
    #df_related_artists['genere'] = genres_list
    df_related_artists['popularity'] = popularity_list
    df_related_artists['followers'] = followers_list
    df_related_artists['url'] = artist_url_list
    df_related_artists = df_related_artists.sort_values(by=['popularity','followers'], ascending=[False,False])
    
    return df_related_artists

In [138]:
a = get_related_artists('周杰伦')
a

,name,popularity,followers,url
7,JJ Lin,67,1996239,7Dx7RhX0mFuXhCOUgB01uM
5,Eason Chan,65,1583897,2QcZxAgcs2I1q7CtCkl6MI
14,Mayday,65,1194544,16s0YTFcyjP4kgFwt7ktrY
17,Jacky Cheung,63,1090704,1Hu58yHg2CXNfDhlPd7Tdd
2,Jolin Tsai,61,606532,1r9DuPTHiQ7hnRRZ99B8nL
3,Leehom Wang,60,540593,2F5W6Rsxwzg0plQ0w8dSyt
10,A-Lin,59,694261,28gf2piFx6cAKOMIwcky5a
15,Karen Mok,58,389544,6jlz5QSUqbKE4vnzo2qfP1
1,Rainie Yang,57,491240,0MEchSWR9pJvw1S5CV3Kuk
6,S.H.E,56,452487,2lWShxOL8iTlI0pmtVKvEh
